In [ ]:
## Remove all previous preparation data
#!rm -rf prep

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import skimage 
from scipy import ndimage, misc


resize = (64, 64)

In [20]:
from keras.utils import np_utils
from sklearn.datasets import load_files

data = load_files('train')
file_names = np.array(data['filenames'])
target = np.array(data['target'])

In [21]:
len(file_names)

22101

In [22]:
%%time
train_metadata = []

for root, dirs, files in os.walk("./train"):
    for sf in dirs:
        if sf.split('.')[0][-1]=='0':
            print('reading files from {}...'.format(sf))
        for f_name in os.listdir(os.path.join("./train", sf)):
            if not f_name.startswith('.'):
                temp = misc.imread(os.path.join("./train", sf, f_name)) # read image
                # collect image metadata
                image_metadata = []
                image_metadata.extend([sf, f_name, os.path.join("./train", sf, f_name)])
                image_metadata.extend( list(temp.shape) if len(temp.shape) == 3\
                                      else [temp.shape[0], temp.shape[1], 1])
                image_metadata.extend([temp.nbytes, temp.dtype])
                # append image metadata to list
                train_metadata.append(image_metadata)
                
train_metadata = pd.DataFrame(train_metadata)
#train_metadata.head()
train_metadata.columns = ['category','img_name', 'full_path', 'height','width','channels','byte_size','bit_depth']
# select integer target (1-257)
train_metadata['target'] = train_metadata['category'].str.split('.').apply(lambda x: int(x[0]))
# image extension
train_metadata['img_extension'] = train_metadata['img_name'].str.split('.').apply(lambda x: x[1])
# just name without category indexes
train_metadata['category_name'] = train_metadata['category'].str.split('.').apply(lambda x: x[1]).str.lower()

reading files from 240.watch-101...
reading files from 230.trilobite-101...
reading files from 110.hourglass...
reading files from 160.pez-dispenser...
reading files from 140.menorah-101...
reading files from 020.brain-101...
reading files from 180.screwdriver...
reading files from 190.snake...
reading files from 250.zebra...
reading files from 150.octopus...
reading files from 030.canoe...
reading files from 200.stained-glass...
reading files from 080.frog...
reading files from 100.hawksbill-101...
reading files from 010.beer-mug...
reading files from 050.covered-wagon...
reading files from 090.gorilla...
reading files from 120.joy-stick...
reading files from 220.toaster...
reading files from 060.duck...
reading files from 070.fire-extinguisher...
reading files from 210.syringe...
reading files from 130.license-plate...
reading files from 040.cockroach...
reading files from 170.rainbow...
CPU times: user 47.2 s, sys: 368 ms, total: 47.6 s
Wall time: 47.9 s


In [23]:
train_metadata.groupby('category').size().sort_values()

category
223.top-hat                50
086.golden-gate-bridge     50
179.scorpion-101           50
099.harpsichord            50
204.sunflower-101          50
186.skunk                  51
218.tennis-racket          51
082.galaxy                 51
201.starfish-101           51
121.kangaroo-101           52
053.desk-globe             52
144.minotaur               52
146.mountain-bike          52
020.brain-101              53
175.roulette-wheel         53
059.drinking-straw         53
062.eiffel-tower           53
160.pez-dispenser          53
066.ewer-101               53
067.eyeglasses             53
075.floppy-disk            53
184.sheet-music            54
084.giraffe                54
248.yarmulke               54
076.football-helmet        54
215.telephone-box          54
070.fire-extinguisher      54
174.rotary-phone           54
219.theodolite             54
239.washing-machine        54
                         ... 
101.head-phones           108
214.teepee                109
1

In [24]:
def image_preproc_save(path_to_file, path_to_save, resize_dim = (224,224)):
    image = misc.imread(path_to_file)
    if len(image.shape) == 2:
        image = np.stack((image,)*3)
    misc.imsave(path_to_save,misc.imresize(image, size=resize_dim))

In [25]:
from sklearn.model_selection import train_test_split

def val_split(files, target, uniform = 0, test_size = 0.3):
    if uniform == 0:
        X_train, X_val, y_train, y_val = train_test_split(files, target, test_size=0.30, random_state=242, stratify = target)
    else:
        X_train = []
        X_val = []
        for tg in list(set(target)):
            list_to_split = files[target == tg]
            #print ("BBB", list_to_split)
            val = np.random.choice(list_to_split, uniform, replace=False)
            #print("AAA", val)
            X_train += list(set(list_to_split)-set(val))
            #print("CCC", X_train)
            X_val.extend(val)
    
    return X_train, X_val

In [26]:
def preproc_all(files, folder_to_save, resize = (224, 224)):
    for f_name in files:
        load_path = f_name
        
        sf_file = f_name.split('/')
        
        save_path = os.path.join(folder_to_save, sf_file[1], sf_file[2])
 
        if not os.path.exists(os.path.join(folder_to_save, sf_file[1])):
                os.makedirs(os.path.join(folder_to_save, sf_file[1]))
        try:
            image_preproc_save(load_path, save_path, resize)
        except:
            print("Failed_preprocessing for: {}".format(load_path))
        #print(save_path)

In [32]:
X_train, X_val = val_split(file_names, target, uniform = 13)

In [33]:
print(len(X_train))
print(len(X_val))

18773
3328


In [24]:
#!rm -rf prep224

In [34]:
%%time
preproc_all(X_train, os.path.join('prep13', 'train'), resize)

Failed_preprocessing for: train/056.dog/.DS_Store
CPU times: user 1min, sys: 776 ms, total: 1min 1s
Wall time: 1min 1s


In [35]:
%%time
preproc_all(X_val, os.path.join('prep13', 'val'))

CPU times: user 17.7 s, sys: 124 ms, total: 17.8 s
Wall time: 17.8 s


### Test files

In [36]:
%%time
os.makedirs('prep13/test')
for f_name in os.listdir('test'):
    if not f_name.startswith('.'):
        load_path = os.path.join('test', f_name)
        save_path = os.path.join('prep13', 'test', f_name)
        #try:
        image_preproc_save(load_path, save_path, resize)
        #except:
            #print("Failed_preprocessing for: {}".format(load_path))

CPU times: user 26.4 s, sys: 300 ms, total: 26.7 s
Wall time: 26.8 s
